In [2]:
import torch

In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [5]:
encoded_hello

[61, 58, 65, 65, 68]

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [8]:
type(data)

torch.Tensor

In [11]:
block_size = 8
batch_size = 4

In [16]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([113689, 184853, 180292,  80245])
inputs:
tensor([[ 1, 66, 68, 67, 58, 78,  1, 76],
        [71,  1, 73, 68,  1, 71, 58, 54],
        [58, 54, 65,  1, 73, 68,  1, 61],
        [75, 58,  1, 61, 54, 57,  1, 58]], device='cuda:0')
targets:
tensor([[66, 68, 67, 58, 78,  1, 76, 62],
        [ 1, 73, 68,  1, 71, 58, 54, 57],
        [54, 65,  1, 73, 68,  1, 61, 62],
        [58,  1, 61, 54, 57,  1, 58, 67]], device='cuda:0')


In [15]:
block_size = 8
x = train_data[:block_size]
y = train_data[1: block_size+1]
    
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda
